In [1]:
# Import the functions used in this project
import awesome_functions as af

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
train = pd.read_csv("../asset/train.csv")
test = pd.read_csv("../asset/test.csv")
submission = pd.read_csv("../asset/sample_submission.csv")

# Success - Display the first record
print("Train data : ", train.shape)
print("Test  data : ", test.shape)
print("submission  data : ", submission.shape)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle as pkl
slack_url = pkl.load(open("Slack_url/send_url.pickle", "rb"))

import xgboost

# 원본을 유지하기 위해서 카피
df_train = train.copy()
df_test = test.copy()
df_submission = submission.copy()

Train data :  (647054, 7)
Test  data :  (653646, 6)
submission  data :  (95674, 39)


In [2]:
df_train.Upc = df_train.Upc.astype(str)

In [3]:
df_train.Upc = df_train.Upc.apply(lambda a : a.split(".")[0] if a != "nan" else "000000000000")

In [24]:
df_train.Upc = df_train.Upc.apply(lambda a : a[:-6])

In [6]:
df_train["Upc_length"] =  df_train.Upc.apply(lambda a : len(a))

In [8]:
df_train.drop(["DepartmentDescription", "FinelineNumber", "Weekday"], axis = 1, inplace = True)

In [11]:
tmp_1 = df_train.groupby(["VisitNumber", "Upc"]).agg({"ScanCount" : np.sum})

In [14]:
tmp_1  = tmp_1.reset_index()

In [15]:
upc_li = []
scancount_li = []
for i, vn in enumerate(tmp_1.VisitNumber.unique()):
    tmp_df = tmp_1[tmp_1.VisitNumber == vn]
    max_count = tmp_df["ScanCount"].max()
    upc = tmp_df[tmp_df["ScanCount"] == max_count]["Upc"].values[0]
    upc_li.append(upc)
    scancount_li.append(max_count)
    if i % 5000 == 0:
        print(str(i) + "까지 진행됨")

0까지 진행됨
5000까지 진행됨
10000까지 진행됨
15000까지 진행됨
20000까지 진행됨
25000까지 진행됨
30000까지 진행됨
35000까지 진행됨
40000까지 진행됨
45000까지 진행됨
50000까지 진행됨
55000까지 진행됨
60000까지 진행됨
65000까지 진행됨
70000까지 진행됨
75000까지 진행됨
80000까지 진행됨
85000까지 진행됨
90000까지 진행됨
95000까지 진행됨


In [20]:
df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")

In [50]:
df_mf_upc_sc = pd.DataFrame({"MF_Upc" : upc_li, "ScanCount" : scancount_li})

In [52]:
af.saveDataFrameToCsv(df_mf_upc_sc, "df_mf_upc_sc")

In [21]:
cols = ["VisitNumber", "TripType"]
df_new_upc = pd.concat([df_train_dd[cols], df_mf_upc_sc], axis = 1)

In [32]:
df_new_upc["Upc_len"] = df_new_upc["MF_Upc"].apply(lambda a : len(a))

In [29]:
df_new_upc["MF_Upc"] = df_new_upc["MF_Upc"].apply(lambda a : a if len(a) >= 8 else "777777777777")

In [34]:
df_new_upc["MF_Company"] = df_new_upc["MF_Upc"].apply(lambda a : a[:-6])

In [38]:
np_upc_li = np.array(upc_li)

In [46]:
np_upc_li = np_upc_li.astype(int)

In [53]:
af.saveDataFrameToCsv(df_new_upc, "df_new_upc")

In [33]:
df_new_upc.groupby("Upc_len").size().reset_index(name="Count")

,Upc_len,Count
0,8,49
1,9,171
2,10,62852
3,11,27515
4,12,5087


In [126]:
sc_total = pd.read_csv("Feature_matrix/scancount_total_201808012316.csv")

In [9]:
df_train_ = df_train.copy()

In [10]:
df_train_["FinelineNumber"] = df_train_["FinelineNumber"].astype(str)

In [11]:
df_train_["FinelineNumber"] = df_train_["FinelineNumber"].apply(lambda a : a.split(".")[0] if a != "nan" else "000000000000")

In [12]:
tmp_1 = df_train_.groupby(["VisitNumber", "FinelineNumber"]).agg({"ScanCount" : np.sum})

In [13]:
tmp_1  = tmp_1.reset_index()

In [14]:
fn_li = []
scancount_li = []
for i, vn in enumerate(tmp_1.VisitNumber.unique()):
    tmp_df = tmp_1[tmp_1.VisitNumber == vn]
    max_count = tmp_df["ScanCount"].max()
    fineline = tmp_df[tmp_df["ScanCount"] == max_count]["FinelineNumber"].values[0]
    fn_li.append(fineline)
    scancount_li.append(max_count)
    if i % 5000 == 0:
        print(str(i) + "까지 진행됨")

0까지 진행됨
5000까지 진행됨
10000까지 진행됨
15000까지 진행됨
20000까지 진행됨
25000까지 진행됨
30000까지 진행됨
35000까지 진행됨
40000까지 진행됨
45000까지 진행됨
50000까지 진행됨
55000까지 진행됨
60000까지 진행됨
65000까지 진행됨
70000까지 진행됨
75000까지 진행됨
80000까지 진행됨
85000까지 진행됨
90000까지 진행됨
95000까지 진행됨


In [15]:
df_mf_sc = pd.DataFrame({"MF_FinelineNumber" : fn_li, "ScanCount" : scancount_li})

In [17]:
af.saveDataFrameToCsv(df_mf_sc, "df_mf_sc")

In [16]:
df_mf_sc

,MF_FinelineNumber,ScanCount
0,1000,-1
1,4504,1
2,1017,5
3,103,1
4,115,1
5,110,1
6,203,1
7,514,3
8,3352,1
9,8904,2


In [18]:
cols = ["VisitNumber", "TripType"]

In [19]:
df_new = pd.concat([df_train_dd[cols], df_mf_sc], axis = 1)

In [2]:
sc_total = pd.read_csv("Feature_matrix/scancount_total_201808012316.csv")

In [21]:
df_new["Ratio"] = df_new["ScanCount"].div(sc_total.Total, axis = 0)

In [33]:
sc_total.loc[40265]

Total    1
Name: 40265, dtype: int64

In [34]:
df_train[df_train.VisitNumber == 80542]

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
270466,35,80542,Wednesday,8.768400e+09,1,DSD GROCERY,4624.0
270467,35,80542,Wednesday,8.768400e+09,-9,DSD GROCERY,4624.0
270468,35,80542,Wednesday,6.827474e+09,5,DSD GROCERY,2351.0
270469,35,80542,Wednesday,8.768400e+09,2,DSD GROCERY,4624.0
270470,35,80542,Wednesday,8.768400e+09,-1,DSD GROCERY,4624.0


In [40]:
sc_total = list(df_train.groupby("VisitNumber").sum()["ScanCount"].values)

In [41]:
sc_total

[-1,
 2,
 28,
 3,
 3,
 4,
 7,
 9,
 4,
 9,
 3,
 2,
 3,
 12,
 8,
 1,
 1,
 2,
 1,
 3,
 3,
 3,
 1,
 4,
 4,
 5,
 5,
 1,
 2,
 1,
 1,
 11,
 2,
 1,
 12,
 5,
 1,
 4,
 3,
 2,
 4,
 9,
 1,
 10,
 2,
 2,
 26,
 1,
 1,
 3,
 1,
 1,
 14,
 50,
 2,
 1,
 1,
 1,
 4,
 2,
 1,
 2,
 1,
 2,
 8,
 1,
 1,
 7,
 1,
 0,
 2,
 2,
 7,
 2,
 43,
 5,
 2,
 22,
 2,
 5,
 3,
 2,
 2,
 2,
 4,
 2,
 -1,
 4,
 1,
 2,
 -1,
 16,
 2,
 4,
 2,
 4,
 3,
 2,
 9,
 4,
 7,
 6,
 5,
 1,
 1,
 4,
 1,
 2,
 18,
 3,
 5,
 2,
 2,
 1,
 3,
 1,
 3,
 2,
 8,
 12,
 6,
 13,
 -1,
 55,
 1,
 2,
 15,
 -1,
 3,
 2,
 3,
 20,
 6,
 1,
 6,
 1,
 -1,
 30,
 1,
 6,
 2,
 35,
 1,
 1,
 2,
 5,
 1,
 1,
 2,
 9,
 3,
 2,
 1,
 3,
 3,
 5,
 21,
 4,
 -1,
 2,
 2,
 2,
 2,
 6,
 2,
 4,
 0,
 1,
 1,
 17,
 1,
 1,
 22,
 1,
 1,
 3,
 1,
 2,
 2,
 48,
 2,
 2,
 5,
 8,
 1,
 1,
 1,
 6,
 5,
 2,
 7,
 1,
 15,
 30,
 2,
 8,
 1,
 30,
 12,
 5,
 1,
 3,
 1,
 8,
 9,
 5,
 8,
 4,
 4,
 -1,
 3,
 -1,
 -1,
 9,
 -1,
 2,
 5,
 1,
 1,
 3,
 22,
 2,
 22,
 5,
 7,
 1,
 1,
 -1,
 6,
 2,
 1,
 4,
 2,
 15,
 2,
 7,
 5,
 4,
 4,
 5

In [42]:
df_new = df_new.drop("Ratio", axis = 1)

In [44]:
df_new["Total"] = sc_total

In [46]:
df_new["Ratio"] = df_new["ScanCount"].div(df_new.Total, axis = 0)

In [63]:
af.saveDataFrameToCsv(df_new, "df_new")

In [69]:
df_new_1 = df_new[df_new["Ratio"] != np.inf]

In [72]:
df_new_1 = df_new_1.dropna()

In [106]:
af.saveDataFrameToCsv(df_new, "df_new_has_all_the_ratio")

In [108]:
df_train_dd

,VisitNumber,TripType,Return,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,5,999,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,7,30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,8,26,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,9,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,10,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,11,35,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,12,41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,15,21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,17,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9,19,42,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [109]:
new_cols = ["MF_FinelineNumber", "ScanCount", "Total", "Ratio"]

In [114]:
df_train_dd_imporved = pd.concat([df_train_dd, df_new[new_cols]], axis = 1)

In [115]:
af.saveDataFrameToCsv(df_train_dd_imporved, "df_train_dd_imporved")

In [105]:
df_new[np.isnan(df_new.Ratio)]

,VisitNumber,TripType,MF_FinelineNumber,ScanCount,Total,Ratio
69,133,999,1180,0,0,NaN
166,333,999,6812,0,0,NaN
298,581,999,2064,0,0,NaN
345,685,999,2996,0,0,NaN
390,781,999,8294,0,0,NaN
474,950,999,975,0,0,NaN
567,1120,999,1590,0,0,NaN
694,1368,999,3000,0,0,NaN
743,1460,999,8615,0,0,NaN
744,1466,999,21,0,0,NaN


In [80]:
df_tmp = df_new_1[df_new_1["Total"] > 0]

In [96]:
df_tmp[df_tmp["Ratio"] == 1]

,VisitNumber,TripType,MF_FinelineNumber,ScanCount,Total,Ratio
15,29,8,9205,1,1,1.0
16,30,9,1180,1,1,1.0
18,32,8,100,1,1,1.0
22,42,8,115,1,1,1.0
27,50,9,8218,1,1,1.0
29,53,8,1201,1,1,1.0
30,54,8,2286,1,1,1.0
33,57,8,1026,1,1,1.0
36,68,9,8225,1,1,1.0
39,74,8,2286,2,2,1.0


In [95]:
df_tmp[df_tmp["Ratio"] == 1].groupby("TripType").size().reset_index(name ="count").sort_values("count", ascending=False)

,TripType,count
5,8,5560
6,9,4969
2,5,2225
4,7,1254
34,999,950
0,3,855
3,6,565
23,32,367
13,22,287
15,24,270


In [208]:
pd.concat([df_new, sc_total], axis=1)

,VisitNumber,TripType,MF_FinelineNumber,ScanCount,Total
0,5,999,1000,-1,1
1,7,30,4504,1,3
2,8,26,1017,5,29
3,9,8,103,1,4
4,10,8,115,1,4
5,11,35,110,1,5
6,12,41,203,1,8
7,15,21,514,3,10
8,17,6,3352,1,5
9,19,42,8904,2,10


In [195]:
df_mf_sc

,MF_FinelineNumber,ScanCount
0,1000,-1
1,4504,1
2,1017,5
3,103,1
4,115,1
5,110,1
6,203,1
7,514,3
8,3352,1
9,8904,2


In [187]:
fn_copy = fn_li.copy()

In [188]:
len(set(fn_copy))

4251

In [166]:
len(fn_li)

94247

In [100]:
tmp.Upc = tmp.Upc.astype(str)

In [101]:
tmp.Upc = tmp.Upc.apply(lambda a : a.split(".")[0] if a != "nan" else "000000000000")

In [124]:
tmp[tmp["Upc_length"] < 7].groupby("DepartmentDescription").size().reset_index(name = "Count").sort_values("Count", ascending=False)

,DepartmentDescription,Count
36,PRODUCE,2170
15,GROCERY DRY GOODS,245
11,DSD GROCERY,216
10,DAIRY,101
2,BAKERY,85
13,FROZEN FOODS,69
32,PETS AND SUPPLIES,64
31,PERSONAL CARE,57
20,HOUSEHOLD CHEMICALS/SUPP,34
22,IMPULSE MERCHANDISE,31


In [102]:
tmp["Upc_length"] = tmp.Upc.apply(lambda a : len(a))

In [103]:
tmp.groupby("Upc_length").size().reset_index()

,Upc_length,0
0,3,2
1,4,3211
2,5,74
3,8,83
4,9,365
5,10,56361
6,11,32517
7,12,3061


In [2]:
from sklearn.naive_bayes import MultinomialNB
def fitNaiveBayesModel(X, y):
    return MultinomialNB().fit(X, y)
model_xgb = xgboost.XGBClassifier(n_estimators=100, max_depth=2)

In [3]:
from sklearn.metrics import log_loss

In [4]:
df_train_dd = pd.read_csv("Feature_matrix/df_train_dd_201807291831.csv")
df_test_dd = pd.read_csv("Feature_matrix/df_test_dd_201807291855.csv")

In [5]:
df_train_fl = pd.read_csv("Feature_matrix/df_train_fl_201807291845.csv")
df_test_fl = pd.read_csv("Feature_matrix/df_test_fl_201807291908.csv")

In [6]:
nor_train_fl = pd.read_csv("Feature_matrix/Normalized_train_fl_201808020202.csv")

In [7]:
nor_train_dd = pd.read_csv("Feature_matrix/Normalized_train_dd_201808012338.csv")

In [8]:
df_fl_sc = df_train.groupby(["FinelineNumber"]).sum()["ScanCount"].reset_index().sort_values("ScanCount", ascending =False)
# bottom = 10
# top = 124
# df_fineline_50 = df_fl_sc[(df_fl_sc["ScanCount"]>=bottom)&(df_fl_sc["ScanCount"]<=top)]

In [9]:
df_fineline_0 = df_fl_sc[(df_fl_sc["ScanCount"]>=0)&(df_fl_sc["ScanCount"]<=10)]

In [10]:
df_fineline_75 = df_fl_sc[df_fl_sc["ScanCount"]>124]

In [84]:
cols_75 = list(df_fineline_75["FinelineNumber"].unique())

In [11]:
tmp_df_0 = pd.concat([df_fineline_0, df_fineline_75], axis = 0)

In [12]:
tmp_df_0["FinelineNumber"] = tmp_df_0["FinelineNumber"].astype(str)

In [58]:
cols = list(tmp_df_0["FinelineNumber"].unique())

In [14]:
def getGroupByDf(df):
    cols = [col for col in df.columns if col not in ["VisitNumber", "TripType"]]
    npsum = [np.sum for col in cols]
    agg_dict = dict(zip(cols, npsum))
    return df.groupby("TripType").agg(agg_dict).reset_index()

def getColsToFit(df, is_need_return = True):
    if is_need_return:
        return [col for col in df.columns if col not in ["VisitNumber", "TripType"]]
    return [col for col in df.columns if col not in ["VisitNumber", "TripType", "Return"]]

In [15]:
li_train_by_vn = []
for i in df_train_dd["TripType"].unique():
    tmp = df_train_dd[df_train_dd["TripType"] == i]
    li_train_by_vn.append(tmp)
    
length_train_by_triptype = []
triptype_li = []
for i in range(len(li_train_by_vn)):
    length_train_by_triptype.append(len(li_train_by_vn[i]))
    triptype_li.append(li_train_by_vn[i]["TripType"].unique()[0])

In [16]:
def getNormalizedDf(df, cols):
    result = df.copy()
    result = result[cols]
    for i in range(len(df)):
        triptype = df.loc[i]["TripType"]
        length = dict_len_by_tt[triptype]
        result.loc[i] = round(result.loc[i]/length, 2)
    return result

In [17]:
cols_fl = getColsToFit(df_train_fl, is_need_return = True)
cols_dd = getColsToFit(df_train_dd, is_need_return = True)

In [18]:
group_nor_fl = getGroupByDf(nor_train_fl)

In [19]:
group_nor_dd = getGroupByDf(nor_train_dd)

In [20]:
# df_dd_fl = df_train.groupby(["TripType", "DepartmentDescription", "FinelineNumber"]).sum()["ScanCount"].reset_index()

In [21]:
# li_by_tt = []
# for tt in df_dd_fl["TripType"].unique():
#     tmp = df_dd_fl[df_dd_fl["TripType"] == tt]
#     li_by_tt.append(tmp)

In [28]:
df_fineline_50["FinelineNumber"] = df_fineline_50["FinelineNumber"].astype(str)

In [29]:
tmp_cols = list(df_fineline_50["FinelineNumber"].unique())

In [32]:
len(df_train_fl[tmp_cols].columns)

2607

In [35]:
tmp_df_cols = df_train_fl[["VisitNumber", "Return", "TripType"]]

In [42]:
df_new = pd.concat([df_train_dd, df_train_fl[tmp_cols]], axis=1)

In [22]:
df_new_2 = pd.concat([df_train_dd, df_train_fl[cols]], axis = 1)

In [85]:
df_new_75 = pd.concat([df_train_dd, df_test_fl[cols_75]], axis = 1)

KeyError: '[5501. 1508.  808. ... 4473. 4159. 9189.] not in index'

In [61]:
df_test_new = pd.concat([df_test_dd, df_test_fl[cols]], axis = 1)

In [24]:
rm_cols = ["VisitNumber", "TripType"]

In [63]:
X_test_new = af.get_df_to_fit(df_test_new, True)

In [25]:
cols = [col for col in df_new_2.columns if col not in rm_cols]

In [27]:
X = df_new_2[cols]

In [28]:
y = df_train_dd["TripType"]

In [32]:
from scipy.sparse import csr_matrix

In [33]:
csr_train = csr_matrix(X)

In [34]:
csr_train

<95674x2717 sparse matrix of type '<class 'numpy.int64'>'
	with 877044 stored elements in Compressed Sparse Row format>

In [60]:
len(df_new_2.columns)

2719

In [37]:
df_fl_50 = pd.concat([tmp_df_cols, df_train_fl[tmp_cols]],axis=1)

In [38]:
X, y = af.get_df_to_fit(df_fl_50)

In [29]:
model_nb = fitNaiveBayesModel(X, y)

In [30]:
y_pred = model_nb.predict(X)

In [31]:
af.getAccuracy(y, y_pred, len(X))

##### Accuracy : 0.6104

In [43]:
X_new, y = af.get_df_to_fit(df_new)

In [44]:
model_nb_new = fitNaiveBayesModel(X_new, y)

In [45]:
y_pred_new = model_nb_new.predict(X_new)

In [46]:
af.getAccuracy(y, y_pred_new, len(X))

##### Accuracy : 0.6073

In [47]:
y_pred_proba = model_nb_new.predict_proba(X_new)

In [48]:
log_loss(y, y_pred_proba)

1.8114219488350272

In [61]:
X_new2, y = af.get_df_to_fit(df_new_2)

In [62]:
model_nb_new2 = fitNaiveBayesModel(X_new2, y)

In [64]:
y_pred_new2 = model_nb_new2.predict(X_new2)

In [65]:
af.getAccuracy(y, y_pred_new2, len(X))

##### Accuracy : 0.6104

In [42]:
model_xgb = xgboost.XGBClassifier(max_depth=3, learning_rate=0.5, n_estimators=100,\
                                  eval_metric="logloss",silent=True, objective='binary:logistic', booster='gbtree',\
                                  n_jobs=4, nthread=None, gamma=0, min_child_weight=1,\
                                  max_delta_step=0, subsample=1, colsample_bytree=1, \
                                  colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,\
                                  base_score=0.5, random_state=0, seed=None, missing=None)

In [43]:
model_xgb.fit(csr_train, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=17, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=4, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [44]:
y_pred = model_xgb.predict(csr_train)

In [45]:
af.getAccuracy(y, y_pred, len(X))

##### Accuracy : 0.8856

In [46]:
y_pred_proba = model_xgb.predict_proba(csr_train)

In [ ]:
y_pred_proba

In [ ]:
y_pred_proba_test

In [70]:
X_test_new = X_test_new.drop("index", axis=1)

In [71]:
csr_test = csr_matrix(X_test_new)

In [72]:
csr_test

<95674x2717 sparse matrix of type '<class 'numpy.float64'>'
	with 882383 stored elements in Compressed Sparse Row format>

In [73]:
y_pred_proba_test = model_xgb.predict_proba(csr_test)

In [76]:
submit_new = af.make_submission_df(submission, y_pred_proba_test)

In [77]:
af.saveDataFrameToCsv(submit_new, "submit_new_df", is_submission_df=True)

In [47]:
log_loss(y, y_pred_proba)

0.49790989986149525

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
Y_train = lb.fit_transform(y_train)
Y_test = lb.fit_transform(y_test)

# y_pred = lb.inverse_transform(np.round(predictions))
# y_true = lb.inverse_transform(train_y)

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adagrad
from keras.utils import np_utils

Using TensorFlow backend.


In [94]:
model1 = Sequential()
model1.add(Dense(80, input_dim=2717, activation="sigmoid", kernel_initializer="glorot_uniform"))
model1.add(Dense(38, activation="softmax"))
model1.compile(optimizer=Adagrad(lr=1), loss='categorical_crossentropy', metrics=["accuracy"])

In [95]:
%%time
hist1 = model1.fit(X_train, Y_train, epochs=100, batch_size=1000, validation_data=(X_test, Y_test), verbose=2)

Train on 57404 samples, validate on 38270 samples
Epoch 1/100
 - 8s - loss: 3.3810 - acc: 0.4799 - val_loss: 1.2954 - val_acc: 0.6166
Epoch 2/100
 - 8s - loss: 1.0548 - acc: 0.6785 - val_loss: 1.1621 - val_acc: 0.6432
Epoch 3/100
 - 9s - loss: 0.8344 - acc: 0.7349 - val_loss: 1.0991 - val_acc: 0.6702
Epoch 4/100
 - 8s - loss: 0.7069 - acc: 0.7758 - val_loss: 1.0944 - val_acc: 0.6769
Epoch 5/100
 - 8s - loss: 0.6185 - acc: 0.8054 - val_loss: 1.1122 - val_acc: 0.6844
Epoch 6/100
 - 9s - loss: 0.5521 - acc: 0.8275 - val_loss: 1.1263 - val_acc: 0.6837
Epoch 7/100
 - 8s - loss: 0.4985 - acc: 0.8467 - val_loss: 1.1479 - val_acc: 0.6843
Epoch 8/100
 - 8s - loss: 0.4567 - acc: 0.8599 - val_loss: 1.1745 - val_acc: 0.6843
Epoch 9/100
 - 9s - loss: 0.4222 - acc: 0.8727 - val_loss: 1.2101 - val_acc: 0.6869
Epoch 10/100
 - 9s - loss: 0.3918 - acc: 0.8823 - val_loss: 1.2394 - val_acc: 0.6859
Epoch 11/100
 - 9s - loss: 0.3670 - acc: 0.8912 - val_loss: 1.2749 - val_acc: 0.6831
Epoch 12/100
 - 9s - los

Epoch 97/100
 - 7s - loss: 0.0853 - acc: 0.9732 - val_loss: 2.3793 - val_acc: 0.6491
Epoch 98/100
 - 7s - loss: 0.0847 - acc: 0.9733 - val_loss: 2.3856 - val_acc: 0.6467
Epoch 99/100
 - 7s - loss: 0.0843 - acc: 0.9735 - val_loss: 2.3910 - val_acc: 0.6487
Epoch 100/100
 - 7s - loss: 0.0836 - acc: 0.9736 - val_loss: 2.4010 - val_acc: 0.6453
CPU times: user 17min 42s, sys: 1min 25s, total: 19min 8s
Wall time: 12min 40s
